In [1]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [3]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(
        env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = \
        filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break
writer.close()

C:\Users\AS\AppData\Local\Temp/ipykernel_17092/3210218304.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  obs_v = torch.FloatTensor([obs])


0: loss=0.694, reward_mean=20.2, rw_bound=18.0
1: loss=0.674, reward_mean=24.6, rw_bound=24.0
2: loss=0.660, reward_mean=31.2, rw_bound=34.5
3: loss=0.665, reward_mean=30.7, rw_bound=28.0
4: loss=0.650, reward_mean=40.6, rw_bound=49.0
5: loss=0.627, reward_mean=48.9, rw_bound=47.5
6: loss=0.641, reward_mean=44.4, rw_bound=52.5
7: loss=0.616, reward_mean=39.2, rw_bound=51.5
8: loss=0.610, reward_mean=41.6, rw_bound=47.0
9: loss=0.622, reward_mean=38.3, rw_bound=42.5
10: loss=0.599, reward_mean=51.4, rw_bound=58.5
11: loss=0.583, reward_mean=46.3, rw_bound=52.0
12: loss=0.581, reward_mean=55.3, rw_bound=58.0
13: loss=0.572, reward_mean=54.6, rw_bound=61.5
14: loss=0.574, reward_mean=52.1, rw_bound=65.0
15: loss=0.563, reward_mean=62.7, rw_bound=69.5
16: loss=0.558, reward_mean=65.6, rw_bound=77.0
17: loss=0.557, reward_mean=66.1, rw_bound=72.0
18: loss=0.528, reward_mean=76.9, rw_bound=83.5
19: loss=0.540, reward_mean=69.0, rw_bound=78.5
20: loss=0.553, reward_mean=64.3, rw_bound=79.0
21